Import Libraries

In [1]:
import argparse, pathlib, numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
device = 'cuda:0' if torch.cuda.is_available() else "cpu"
print(f'Device is {device}')

Device is cuda:0


In [3]:
data_transforms = {
 'train': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
 'val': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
 'test': transforms.Compose([
 transforms.Resize((224,224)),
 transforms.ToTensor(),
 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ]),
}

In [5]:
# unzip the  file
import zipfile
with zipfile.ZipFile("hw2_data.zip","r") as zip_ref:
    zip_ref.extractall("hw2_data")

In [6]:
data_dir = '/content/hw2_data/hw2_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
 for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4)
 for x in ['train', 'val' , 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [7]:
class VGG16_Feature_Extraction(torch.nn.Module):
  def __init__(self):
    super(VGG16_Feature_Extraction, self).__init__()
    VGG16_Pretrained = models.vgg16(pretrained=True)
    self.features = VGG16_Pretrained.features
    self.avgpool = VGG16_Pretrained.avgpool
    self.feature_extractor = nn.Sequential(*[VGG16_Pretrained.classifier[i] for i in range(6)])
  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.feature_extractor(x)
    return x

In [8]:
model = VGG16_Feature_Extraction()
model = model.to(device)


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [9]:
print('Extracting Features')
image_features = {}
image_labels = {}
for phase in ['train', 'test']:
 for inputs, labels in dataloaders[phase]:
  inputs = inputs.to(device)
  model_prediction = model(inputs)
  model_prediction_numpy = model_prediction.cpu().detach().numpy()
  if (phase not in image_features):
   image_features[phase] = model_prediction_numpy
   image_labels[phase] = labels.numpy()
  else:
   image_features[phase] = np.concatenate((image_features[phase], model_prediction_numpy), axis=0)
   image_labels[phase] = np.concatenate((image_labels[phase], labels.numpy()), axis=0)

Extracting Features


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
import pickle as pickle
pickle_out = open("/content/pickle/image_features.pickle","wb")
pickle.dump(image_features, pickle_out)
pickle_out.close()

In [12]:
pickle_out = open("/content/pickle/image_labels.pickle","wb")
pickle.dump(image_labels, pickle_out)
pickle_out.close()

In [13]:
print('Training SVM')
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0))
clf.fit(image_features['train'], image_labels['train'])



Training SVM


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=0,
                           tol=0.0001, verbose=0))],
         verbose=False)

In [14]:
print('Testing SVM')

y_true = image_labels['test']
y_pred = clf.predict(image_features['test'])
print("Part A Accuracy Score: ",{accuracy_score(y_true, y_pred)})
print("Part A Confusion Matrix:\n",np.array2string(confusion_matrix(y_true, y_pred)))

Testing SVM
Part A Accuracy Score:  {0.5084033613445378}
Part A Confusion Matrix:
 [[22  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0  0  1  0]
 [ 0 13  0  1  0  1  1  0  0  0  0  0  0  4  2  1  0  1  0  1]
 [ 0  0 17  2  0  0  0  1  1  0  0  0  0  0  0  1  1  1  0  1]
 [ 0  2  0 20  2  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  6  0  2  0  2  0  7  0  0  0  3  1  0  1  0  2]
 [ 0  0  0  0  0 15  1  0  1  0  1  0  0  1  1  0  1  0  0  0]
 [ 1  1  0  1  0  1 10  0  1  0  1  0  0  1  3  1  0  1  1  2]
 [ 0  0  0  1  2  0  0 17  1  1  0  2  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  6  0  2  1  4  0  6  2  0  0  1  1  0  1  0  1]
 [ 0  0  0  0  0  0  0  0  1  3  0  1  1  1  3  3  2  0  0  0]
 [ 1  0  0  0  5  0  0  1  2  0  8  0  0  0  5  1  0  1  0  1]
 [ 0  0  0  2  1  0  1  1  0  1  1  9  0  0  1  0  2  4  0  2]
 [ 1  1  0  0  0  0  1  0  1  2  0  1 14  0  1  0  0  0  0  1]
 [ 0  0  0  1  0  0  1  0  0  0  0  0  0 22  0  0  0  0  1  0]
 [ 0  0  0  1  6  0  2  0  1  0  1 